In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import gzip as gz
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

## Basic TensorFlow usage

In [ ]:
a = tf.constant([1, 2])
b = tf.constant([3, 4])
print(a + b)

In [ ]:
# convenience function to read EMNIST data into numpy array
def read_mnist(images_path, labels_path):
    with gz.open(labels_path, 'rb') as labelsFile:
        labels = np.frombuffer(labelsFile.read(), dtype=np.uint8, offset=8)

    with gz.open(images_path,'rb') as imagesFile:
        length = len(labels)
        # Load flat 28x28 px images (784 px), and convert them to 28x28 px
        features = np.frombuffer(imagesFile.read(), dtype=np.uint8, offset=16) \
                        .reshape(length, 784) \
                        .reshape(length, 28, 28)
        features = features.astype(float)
        flip = features[:,:, ::-1,...]       # note that images are flipped
        features = np.rot90(flip, 1, (1,2))  # and rotated 90deg
        
    return features, labels

## Load Data
Test and training data and labels need to be read from the binary files and converted into a form for easy use. Luckily, this data is small enough to fit in memory.
This data set has some pecularities, like images are flipped and rotated 90%. This needs to be accounts for as well.

In [ ]:
train = {}
test = {}

train['features'], train['labels'] = read_mnist('data/emnist-bymerge-train-images-idx3-ubyte.gz', 'data/emnist-bymerge-train-labels-idx1-ubyte.gz')
test['features'], test['labels'] = read_mnist('data/emnist-bymerge-test-images-idx3-ubyte.gz', 'data/emnist-bymerge-test-labels-idx1-ubyte.gz')

In [ ]:
# Lets read the labels so that directories can be named appropriately
def map_labels(label_mappings="data/emnist-bymerge-mapping.txt"):
    labels_dict = {}
    with open(label_mappings, 'rb') as f:
        # each row of the file has the label first and ascii code next
        for line in f:
            items = line.split()
            # note that data is in bytes, so need to convert
            labels_dict.update({int(items[0]): chr(int(items[1]))})
    return labels_dict

# convenience function to display a grid of random images
def display_images(features, labels, mapping):
    # Now, lets try and generate some sample images to see what the data looks like
    fig=plt.figure(figsize=(9, 9))  # show 8in X 8in image
    columns = 4  # 4 images per row
    rows = 5  # lay out images on 5 rows
    for i in range(1, columns*rows +1):
        img_id = np.random.randint(0, features.shape[0])  # max number of images, from prev section
        fig.add_subplot(rows, columns, i)
        img_data = features[img_id].squeeze()  #,:,:]
        plt.title('Label: %d Char: %s' % ( labels[img_id], mapping[labels[img_id]]))
        plt.imshow(img_data, cmap='gray')
    plt.subplots_adjust(hspace=0.5)
    plt.show()


## Visualize Data
It is always useful to plot a few samples of the data, especially for visual applications, to build intuition and ensure things are as expected.

In [ ]:
labels = map_labels()
display_images(train['features'], train['labels'], labels)

## Our First Model
Keras makes it really easy to specify model architectures at a high level. In TensorFlow 2.0, further simplfications have been made, so that the code is really clean and simple to follow. In the chapter, the model architecture was specified to have 256 units in first hidden layer, 128 in the second one. The hidden layers had ReLU as the activation function and the output layer had softmax.

In [ ]:
model = tf.keras.Sequential(name='First FC DL Model')
# flatten from a 28px X 28px into 784 inputs.
model.add(layers.Flatten(input_shape=(28, 28)))
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(256, activation='relu'))
# Add another:
model.add(layers.Dense(128, activation='relu'))
# Add a softmax layer with 47 output units:
model.add(layers.Dense(47, activation='softmax'))



In [ ]:
# useful to check, especially for complex models
model.summary()

### SideBar: Example of losses


In [ ]:
# Define the values for binary classification. Each element is a prediction
actual = [0., 0., 1., 1.]
predicted = [0.95, 0.88, 1., 0.]

# different loss definitions
mae = tf.keras.losses.MeanAbsoluteError()
mae_loss = mae(actual, predicted)

mse = tf.keras.losses.MeanSquaredError()
mse_loss = mse(actual, predicted)

print('MAE Loss: ', mae_loss.numpy())  # Loss: 0.7075
print('MSE Loss: ', mse_loss.numpy())  # Loss: 0.6692

In [ ]:
# Categorical cross entropy loss
cce = tf.keras.losses.CategoricalCrossentropy()
loss = cce(
  [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]],
  [[.9, .05, .05], [.5, .89, .6], [.05, .01, .94]])
print('Categorical Cross-entropy Loss: ', loss.numpy())  # Loss: 0.3239

# Compile Model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Simple straightforward training
one_hot_labels = tf.one_hot(train['labels'], 47)
history = model.fit(train['features'], one_hot_labels, epochs=10, batch_size=128)

In [ ]:
# Visualize the training
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.legend(['accuracy', 'loss'], loc='upper right')
plt.xlabel('epoch')
plt.show()

In [ ]:
# and evaluation against the test set
one_hot_test = tf.one_hot(test['labels'], 47)
model.evaluate(test['features'], one_hot_test)

## Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  # sklearn has pre-built modules

predictions = model.predict_on_batch(test['features'])  # generates a probability for each label
y_pred = np.argmax(predictions, axis=1)  # choose the one with the highest value as the output label

print('Confusion Matrix')
confusion = confusion_matrix(test['labels'], y_pred)
print(confusion)

### Pretty plotting the confusion matrix

In [ ]:
# convenience function to enable reuse
def plot_confusion_matrix(confusion):
    fig, ax = plt.subplots(figsize=(15, 15))
    im = ax.imshow(keras.utils.normalize(confusion), cmap="YlGn")

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, shrink=0.75)
    cbar.ax.set_ylabel("Percentage of Samples", rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(47))
    ax.set_yticks(np.arange(47))
    # ... and label them with the respective list entries
    ax.set_xticklabels(list(labels.values()))  # NOTE: The axis labels are hard coded here
    ax.set_yticklabels(list(labels.values()))

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)
    ax.set_xticks(np.arange(47+1)-.5, minor=True)
    ax.set_yticks(np.arange(47+1)-.5, minor=True)
    ax.grid(which="minor", color="gray", linestyle='-', linewidth=2)

    fig.tight_layout()

    plt.show()
    
# Lets plot the confusion matrix now
plot_confusion_matrix(confusion)

# Normalization of inputs

In [ ]:
norm_train_features = keras.utils.normalize(train['features'])
norm_test_features = keras.utils.normalize(test['features'])
one_hot_labels = tf.one_hot(train['labels'], 47)

In [ ]:
# Training with normalization of inputs
model.fit(norm_train_features, one_hot_labels, epochs=10, batch_size=256)  # includes a step of data normalization

In [ ]:
# and evaluation
model.evaluate(norm_test_features, tf.one_hot(test['labels'], 47))

In [ ]:
predictions = model.predict_on_batch(test['features'])  # generates a probability for each label
y_pred = np.argmax(predictions, axis=1)  # choose the one with the highest value as the output label

print('Confusion Matrix')
confusion = confusion_matrix(test['labels'], y_pred)
print(confusion)

In [ ]:
plot_confusion_matrix(confusion)

In [ ]:
# model was trained with data normalization and test set was not normalized
# and evaluation
model.evaluate(test['features'], tf.one_hot(test['labels'], 47))

## Precision and recall metrics


In [ ]:
from sklearn.metrics import classification_report

pred = model.predict(norm_test_features, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
report = classification_report(test['labels'], predicted)
print(report)

## Saving and reloading a model

In [ ]:
# lets save the last trained model using data normalization
model.save("emnist_fc_model.h5")

In [ ]:
new_model = keras.models.load_model("emnist_fc_model.h5")
new_model.evaluate(norm_test_features, tf.one_hot(test['labels'], 47))